import packages & selct dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import spatial
import os
import h5py

import mosaic_topog.flsyst as flsyst
import mosaic_topog.show as show
import mosaic_topog.calc as calc

data_path = 'C:\\Users\\schle\Documents\\GitHub\\Incubator-2022-Geometry-of-Color\\CSV data all OCT locs Jan 2022\\'
save_path = 'C:\\Users\\schle\Documents\\Incubator Data\\'

subject = np.asarray(['AO001R', 'AO008R'])
angle = np.asarray(['temporal', 'nasal', 'superior', 'inferior'])
eccentricity = np.asarray(['1.5', '4', '10'])

coord_fltype = '.csv'
img_fltype = '.png'

filetypes = [coord_fltype, img_fltype]

conetype_ids = ['all','L','M','S','unclassed']

conetype_colors = ['y','r','g','b','k']
conetype_sizes = [10, 10, 10, 30]

bin_width = 5

num_mc = 100 # number of monte carlo mosaics to generate

categories = [subject, angle, eccentricity]

load in coordinate data and images

In [2]:
[coord_data, coord_mosaics, coord_index] = flsyst.getConeData(data_path,subject,angle,eccentricity,conetype_ids,filetypes[0])
[img_data, img_mosaics, img_index] = flsyst.getConeData(data_path,subject,angle,eccentricity,conetype_ids,filetypes[1])


found 110 files



c:\users\schle\documents\github\incubator-2022-geometry-of-color\mosaic_topog\mosaic_topog\flsyst.py:88: UserWarning: loadtxt: Empty input file: "C:\Users\schle\Documents\GitHub\Incubator-2022-Geometry-of-Color\CSV data all OCT locs Jan 2022\AO001R_temporal_1.5_unclassed.csv"
  data.append(np.loadtxt(fold_path + fl, delimiter=','))


found 22 files



c:\users\schle\documents\github\incubator-2022-geometry-of-color\mosaic_topog\mosaic_topog\flsyst.py:94: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.asarray(data)


plot cones on the image 

In [ ]:
# for each mosaic we have an image of...
axes = []
for ind,mos in enumerate(img_mosaics):
    # get the image
    img = img_data[ind]
    
    # find the same mosaic in our coordinate data 
    # (may or may not be represented by the same number) 
    coord_mos_ind = coord_mosaics.index(mos)
    
    # pull all coone coordinate filse associated with this mosaic
    coord_mos = flsyst.getDataByCatVal(coord_data, coord_index['mosaics'], coord_mos_ind)
    
    # make a dictionary of all coordinate files and their associated conetype_ids
    coords = {}
    for ind,id in enumerate(conetype_ids):
        coords[id] = coord_mos[ind]
    
    axes.append(show.plotOnROI(img, coords, conetype_ids, mos, conetype_colors, title=mos, figsize=10))
    

make file structure to save calculated data

In [ ]:
# check what experiment numbers have already been run, 
# find the next highest number to name the data save folder
exp_num = 0
satisfied = 0
while not satisfied:
    exp_path = save_path + 'exp_' + str(exp_num) + '\\' 
    try:
        os.mkdir(exp_path)
        satisfied = 1
    except OSError as error: 
        exp_num = exp_num + 1

# start a save file
savename = exp_path+'exp'+str(exp_num)+'.hdf5'
sav = h5py.File(savename, 'w')

print('data will be saved at the following location:')
print(savename)
print('')

calculate the distances between all cones in each mosaic and save them

In [3]:
cone_dists = []

for ind,fl in enumerate(coord_data):     
    cone_dists.append(calc.dist_matrices(coord_data[ind]))

sav.create_dataset(cone_dists, data=cone_dists)

(1470, 2)
(960, 2)
(430, 2)
(80, 2)
(0,)


ValueError: not enough values to unpack (expected 2, got 1)